In [10]:
from math import log, sqrt, pi, exp, isclose, floor
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq
from datetime import datetime, date
import numpy as np
from yahoo_fin import options
from yahoo_fin import stock_info as si

In [2]:
## Black-Scholes Formulas ##
def d1(S, K, t, r, q, vola):
    return(log(S/K)+(r-q+vola**2/2.)*t)/(vola*sqrt(t))

def d2(S, K, t, r, q, vola):
    return d1(S, K, t, r, q, vola)-vola*sqrt(t)

def bs_call(S, K, t, r, q, vola):
    return S*exp(-q*t)*norm.cdf(d1(S, K, t, r, q, vola))-K*exp(-r*t)*norm.cdf(d2(S, K, t, r, q, vola))

def bs_put(S, K, t, r, q, vola):
    return K*exp(-r*t)-S*exp(-q*t)+bs_call(S, K, t ,r, q, vola)

In [3]:
## European option pricers ##

def implied_volatility_european_call(S, K, t, r, q, call_market_price, a=-2.0, b=2.0, xtol=1e-6):
    _S, _K, _t, _r, _q, _call_market_price = S, K, t, r, q, call_market_price
    
    # define a nested function with only volatility as input
    def call_iv_objective_func(vola):
        return _call_market_price - bs_call(_S, _K, _t, _r, _q, vola)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(call_iv_objective_func, a=a, b=b, xtol=xtol)
        
        # if the results are too small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    except ValueError:
        return np.nan
    
def implied_volatility_european_put(S, K, t, r, q, put_market_price, a=-2.0, b=2.0, xtol=1e-6):
    _S, _K, _t, _r, _q, _put_market_price = S, K, t, r, q, put_market_price
    
    # define a nested function with only volatility as input
    def put_iv_objective_func(vola):
        return _put_market_price - bs_put(_S, _K, _t, _r, _q, vola)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(put_iv_objective_func, a=a, b=b, xtol=xtol)
        
        # if the results are too small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    except ValueError:
        return np.nan

#Test functions
S = 45.0
K = 45.0
t = 164.0/365.0
r = 0.02
q = 0.014
vola = 0.25

call_price = bs_call(S, K, t, r, q, vola)
put_price = bs_put(S, K, t, r, q, vola)
print('Make sure that '+ str(implied_volatility_european_call(S, K, t, r, q, call_price)) + ' is close to 0.25')
print('Make sure that '+ str(implied_volatility_european_put(S, K, t, r, q, put_price)) + ' is close to 0.25')

Make sure that 0.24999985507818948 is close to 0.25
Make sure that 0.24999985507818948 is close to 0.25


In [4]:
### American option pricers ###
def binomial_american_call_pricer (S, K, t, t_q, r, q, vola, N):
    u = exp(vola*sqrt(t/N))
    d = exp(-vola*sqrt(t/N))
    R = exp(r*t/N)
    p = (R-d)/(u-d)

    #Create vector where all indices past the ex-div date contain the dividend as % of spot
    dividend = np.zeros(N + 1)

    if t_q <= t and t_q != 0:
        dividend_index = int(floor(t_q/t * N))

        for i in range(N + 1):
            if i >= dividend_index:
                dividend[i] = q


    #Build price tree deducting the dividend
    stock = np.zeros([N + 1, N + 1])
    for i in range(N + 1):
        for j in range(i + 1):
            stock[j, i] = S * (u ** (i - j)) * (d ** j) * (1 - dividend[i])

    # Generate option prices recursively
    option = np.zeros([N + 1, N + 1])
    option[:, N] = np.maximum(np.zeros(N + 1), stock[:, N] - K)

    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option[j, i] = (
                 np.maximum(stock[j, i] - K, (p * option[j, i + 1] + (1 - p) * option[j + 1, i + 1]) / R)
            )

    return option[0, 0]

def binomial_american_put_pricer (S, K, t, t_q, r, q, vola, N):
    u = exp(vola*sqrt(t/N))
    d = exp(-vola*sqrt(t/N))
    R = exp(r*t/N)
    p = (R-d)/(u-d)

    #Create vector where all indices past the ex-div date contain the dividend as % of spot
    dividend = np.zeros(N + 1)

    if t_q <= t and t_q != 0:
        dividend_index = int(floor(t_q/t * N))

        for i in range(N + 1):
            if i >= dividend_index:
                dividend[i] = q


    #Build price tree deducting the dividend
    stock = np.zeros([N + 1, N + 1])
    for i in range(N + 1):
        for j in range(i + 1):
            stock[j, i] = S * (u ** (i - j)) * (d ** j) * (1 - dividend[i])

    # Generate option prices recursively
    option = np.zeros([N + 1, N + 1])
    option[:, N] = np.maximum(np.zeros(N + 1), (K - stock[:, N]))

    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option[j, i] = (
                 np.maximum(K - stock[j, i], (p * option[j, i + 1] + (1 - p) * option[j + 1, i + 1]) / R)
            )

    return option[0, 0]

print(binomial_american_put_pricer(S=100, K=100, vola=0.15, t=1., t_q=1., r=0.1, q=0, N=10))
print(binomial_american_call_pricer(S=100, K=100, vola=0.15, t=1., t_q=1., r=0.1, q=0.0139, N=10))


3.07621086348687
10.87384463741694


In [5]:
## Binomial implied volatility calculation formulas ##

def implied_volatility_american_call(S, K, t, t_q, r, q, N, call_market_price, a=0.01, b=2.0, xtol=1e-6):
    _S, _K, _t, _t_q, _r, _q, _N, _call_market_price = S, K, t, t_q, r, q, N, call_market_price
    
    # define a nested function with only volatility as input
    def call_iv_objective_func(vola):
        return _call_market_price - binomial_american_call_pricer(_S, _K, _t, _t_q, _r, _q, vola, _N)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(call_iv_objective_func, a=a, b=b, xtol=xtol)

        # if the results are too small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    except ValueError:
        print("call error")
        return np.nan
    
def implied_volatility_american_put(S, K, t, t_q, r, q, N, put_market_price, a=0.01, b=2.0, xtol=1e-6):
    _S, _K, _t, _t_q, _r, _q, _N, _put_market_price = S, K, t, t_q, r, q, N, put_market_price
    
    # define a nested function with only volatility as input
    def put_iv_objective_func(vola):
        return _put_market_price - binomial_american_put_pricer(_S, _K, _t, _t_q, _r, _q, vola, _N)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(put_iv_objective_func, a=a, b=b, xtol=xtol)

        # if the results are too small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    except ValueError:
        print("put error")
        return np.nan

#Test functions
S = 45.0
K = 45.0
t = 20/days_in_year
t_q = 20/days_in_year
r = 0.009
q = 0.0139
vola = 0.25
N = 200

call_price = binomial_american_call_pricer(S, K, t, t_q, r, q, vola, N)
put_price = binomial_american_put_pricer(S, K, t, t_q, r, q, vola, N)
print(call_price)
print(put_price)
print('Make sure that '+ str(implied_volatility_american_call(S, K, t, t_q, r, q, N, call_price)) + ' is close to 0.25')
print('Make sure that '+ str(implied_volatility_american_put(S, K, t, t_q, r, q, N, put_price)) + ' is close to 0.25')

1.0599400180692105
1.3730491142518082
Make sure that 0.25000000000342965 is close to 0.25
Make sure that 0.25000001151192275 is close to 0.25


In [22]:
## Market data fetch ##

ticker = "spy"
options_series = datetime(2021, 9, 17)
chain = options.get_options_chain(ticker, options_series)
call_chain = chain["calls"]
put_chain = chain["puts"]
options.get_expiration_dates(ticker);

In [34]:
#Calculate time to expiration
expiration_date = datetime(2021, 9, 17, 16, 0, 0)
ex_dividend_date = datetime(2021, 9, 17)
minutes_in_a_year=365*24*60
time_to_expiration = floor((expiration_date - datetime.now()).total_seconds() / 60.) / minutes_in_a_year
time_to_dividend = floor((ex_dividend_date - datetime.now()).total_seconds() / 60 )/ minutes_in_a_year